In [3]:
from influxdb import InfluxDBClient
import pandas as pd
import os
import datetime
import re
import time
import numpy as np
from datetime import timedelta



連線+撈取

In [17]:
def grab_data(db_name):
    #連線
    name = db_name
    client = InfluxDBClient('120.107.146.56', 8086, 'ncue01', 'Q!A@Z#WSX', name) 
    tables = client.query('show measurements;')
    tables =  list(tables.get_points())
    for table in tables:
        print(table)
        #查看該資料表所有內容
        tablename = table['name']
        result = client.query(f'select * from "{tablename}" where time >= now()-10d') 
         #資料表轉換為list格式
        data =  list(result.get_points())
        data = pd.DataFrame(data)
        if name == 'sensor_db':
            #     判斷是否有該資料夾
            path = f'./Dataset/20230118/dormitory/{tablename}'
            if not os.path.isdir(path):
                os.mkdir(path)
            data.to_csv(f"./Dataset/20230118/dormitory/{tablename}/{tablename}_{datetime.date.today()}.csv", index=False)
        else:
            #     判斷是否有該資料夾
            path = f'./Dataset/20230118/building/{tablename}'
            if not os.path.isdir(path):
                os.mkdir(path)
            data.to_csv(f"./Dataset/20230118/building/{tablename}/{tablename}_{datetime.date.today()}.csv", index=False)

單位轉換

In [2]:
def power_preprocessing(data):
    #將欄位資料從['UpdateTime','kWh']擴充到['UpdateTime','kWh','Hour','Date','power']
    data['UpdateTime'] = pd.to_datetime(data['UpdateTime']) 
    data['Hour'] = data['UpdateTime'].dt.hour
    data['Date'] = data['UpdateTime'].dt.date
    #依據UpdateTime做排序，
    data = data.sort_values(by='UpdateTime')
    #將小時內重複出現的資料刪除，只保留第一筆
    data1 = data.drop_duplicates(subset=['Date','Hour'], keep="first").reset_index(drop=True)
    #依據電表，用下一時段-前一時段
    data1['power'] = data1['kWh'].shift(-1) - data1['kWh']

    return data1

資料合併

In [1]:
def data_merge():
    files = ['building', 'dormitory']
    # files = ['dormitory']
    # 抓取資料夾裡面的資料，進行合併
    for file in files:
        for filename in os.listdir(f"./Dataset/20230118/{file}/"):
            if filename=='merge' or filename=='.ipynb_checkpoints':
                continue
            merge_data = pd.DataFrame()
            for name in os.listdir(f"./Dataset/20230118/{file}/{filename}"):
                if filename=='merge' or filename=='.ipynb_checkpoints':
                    continue
                print(name)
                if name=='.ipynb_checkpoints':
                    continue
                print(f'./Dataset/20230118/{file}/{filename}/{name}')
                try:
                    data = pd.read_csv(f'./Dataset/20230118/{file}/{filename}/{name}')
                except:
                    continue
               
                if 'kWh+' in data.columns:
                    data = data.rename(columns={'kWh+':'kWh'})
                data = data[['UpdateTime', 'kWh']]
                merge_data = pd.concat([merge_data,data], axis=0, ignore_index=True)
            #丟入前處理，並將每棟資料重新存取
            if merge_data.empty == True:
                continue
            else:
                merge_data = power_preprocessing(merge_data)
                merge_data = merge_data.rename(columns={'UpdateTime':'TIME_TO_INTERVAL'})
                merge_data = merge_data.reset_index(drop=True)
                merge_data.to_csv(f'./Dataset/20230118/temp/{filename}.csv', index=False)
            
    redatadic = {}
    for filename in os.listdir(f"./Dataset/20230118/temp"):
        if (filename[-3:]!='csv'):
            continue
        r = "[A-Za-z0-9#&-]"
    #     print(re.sub(r, '', filename[:-4]), filename[:-4])
        #re.sub:用於將匹配的字串取代成指定字串
        #從新命名
        redataname = re.sub(r, '', filename[:-4])
        print(filename)

        data = pd.read_csv(f'./Dataset/20230118/temp/{filename}')
    #     data_nan = grab_breakpoint(data)
    #     data = remove_interruption(data_nan,data)

        #將同名字的館院資料合併
        if redataname not in redatadic.keys():
            redatadic[redataname] = data.copy()
        else:
            redatadic[redataname] = pd.concat([redatadic[redataname], data],axis=0, ignore_index=True)
    #             print(redatadic[redataname])


    # # redatadics = redatadic.copy()
    #
    for d in redatadic.keys():
        print(d)
        #將不同館但同時段的資料合併加總，index重算
        redatadic[d] = redatadic[d].groupby(['Date', 'Hour']).sum().reset_index()
        redatadic[d]['TIME_TO_INTERVAL'] = redatadic[d].apply(lambda x: "{} {:02d}:00:00".format(x['Date'],x['Hour']), axis=1)

    for i in redatadic:
        redatadic[i].to_csv(f'./Dataset/20230118/merge/{i}.csv', index=False)

In [7]:
# grab_data('dorm_db')
# grab_data('sensor_db')
data_merge()

第九宿舍1L1A_第九宿舍1L1A_0.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_0.csv
第九宿舍1L1A_第九宿舍1L1A_10000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_10000.csv
第九宿舍1L1A_第九宿舍1L1A_100000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_100000.csv
第九宿舍1L1A_第九宿舍1L1A_105000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_105000.csv
第九宿舍1L1A_第九宿舍1L1A_110000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_110000.csv
第九宿舍1L1A_第九宿舍1L1A_15000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_15000.csv
第九宿舍1L1A_第九宿舍1L1A_20000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_20000.csv
第九宿舍1L1A_第九宿舍1L1A_25000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_25000.csv
第九宿舍1L1A_第九宿舍1L1A_30000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_30000.csv
第九宿舍1L1A_第九宿舍1L1A_35000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1A_第九宿舍1L1A_35000.csv
第九宿舍1L1A_第九宿舍1L1A_40000.csv
./Dataset/20230118/building/第九宿舍1L1A/第九宿舍1L1

第九宿舍1L3A_第九宿舍1L3A_0.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_0.csv
第九宿舍1L3A_第九宿舍1L3A_10000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_10000.csv
第九宿舍1L3A_第九宿舍1L3A_100000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_100000.csv
第九宿舍1L3A_第九宿舍1L3A_105000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_105000.csv
第九宿舍1L3A_第九宿舍1L3A_110000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_110000.csv
第九宿舍1L3A_第九宿舍1L3A_15000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_15000.csv
第九宿舍1L3A_第九宿舍1L3A_20000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_20000.csv
第九宿舍1L3A_第九宿舍1L3A_25000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_25000.csv
第九宿舍1L3A_第九宿舍1L3A_30000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_30000.csv
第九宿舍1L3A_第九宿舍1L3A_35000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3A_第九宿舍1L3A_35000.csv
第九宿舍1L3A_第九宿舍1L3A_40000.csv
./Dataset/20230118/building/第九宿舍1L3A/第九宿舍1L3

第九宿舍1L5A_第九宿舍1L5A_0.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_0.csv
第九宿舍1L5A_第九宿舍1L5A_10000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_10000.csv
第九宿舍1L5A_第九宿舍1L5A_100000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_100000.csv
第九宿舍1L5A_第九宿舍1L5A_105000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_105000.csv
第九宿舍1L5A_第九宿舍1L5A_110000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_110000.csv
第九宿舍1L5A_第九宿舍1L5A_115000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_115000.csv
第九宿舍1L5A_第九宿舍1L5A_120000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_120000.csv
第九宿舍1L5A_第九宿舍1L5A_125000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_125000.csv
第九宿舍1L5A_第九宿舍1L5A_130000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_130000.csv
第九宿舍1L5A_第九宿舍1L5A_135000.csv
./Dataset/20230118/building/第九宿舍1L5A/第九宿舍1L5A_第九宿舍1L5A_135000.csv
第九宿舍1L5A_第九宿舍1L5A_15000.csv
./Dataset/20230118/building/第九宿舍1L

第九宿舍1L6B_第九宿舍1L6B_75000.csv
./Dataset/20230118/building/第九宿舍1L6B/第九宿舍1L6B_第九宿舍1L6B_75000.csv
第九宿舍1L6B_第九宿舍1L6B_80000.csv
./Dataset/20230118/building/第九宿舍1L6B/第九宿舍1L6B_第九宿舍1L6B_80000.csv
第九宿舍1L6B_第九宿舍1L6B_85000.csv
./Dataset/20230118/building/第九宿舍1L6B/第九宿舍1L6B_第九宿舍1L6B_85000.csv
第九宿舍1L6B_第九宿舍1L6B_90000.csv
./Dataset/20230118/building/第九宿舍1L6B/第九宿舍1L6B_第九宿舍1L6B_90000.csv
第九宿舍1L6B_第九宿舍1L6B_95000.csv
./Dataset/20230118/building/第九宿舍1L6B/第九宿舍1L6B_第九宿舍1L6B_95000.csv
第九宿舍1L7A_第九宿舍1L7A_0.csv
./Dataset/20230118/building/第九宿舍1L7A/第九宿舍1L7A_第九宿舍1L7A_0.csv
第九宿舍1L7A_第九宿舍1L7A_10000.csv
./Dataset/20230118/building/第九宿舍1L7A/第九宿舍1L7A_第九宿舍1L7A_10000.csv
第九宿舍1L7A_第九宿舍1L7A_100000.csv
./Dataset/20230118/building/第九宿舍1L7A/第九宿舍1L7A_第九宿舍1L7A_100000.csv
第九宿舍1L7A_第九宿舍1L7A_105000.csv
./Dataset/20230118/building/第九宿舍1L7A/第九宿舍1L7A_第九宿舍1L7A_105000.csv
第九宿舍1L7A_第九宿舍1L7A_110000.csv
./Dataset/20230118/building/第九宿舍1L7A/第九宿舍1L7A_第九宿舍1L7A_110000.csv
第九宿舍1L7A_第九宿舍1L7A_115000.csv
./Dataset/20230118/building/第九宿舍1L7A/第九宿舍1L

第九宿舍1L8B_第九宿舍1L8B_55000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_55000.csv
第九宿舍1L8B_第九宿舍1L8B_60000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_60000.csv
第九宿舍1L8B_第九宿舍1L8B_65000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_65000.csv
第九宿舍1L8B_第九宿舍1L8B_70000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_70000.csv
第九宿舍1L8B_第九宿舍1L8B_75000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_75000.csv
第九宿舍1L8B_第九宿舍1L8B_80000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_80000.csv
第九宿舍1L8B_第九宿舍1L8B_85000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_85000.csv
第九宿舍1L8B_第九宿舍1L8B_90000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_90000.csv
第九宿舍1L8B_第九宿舍1L8B_95000.csv
./Dataset/20230118/building/第九宿舍1L8B/第九宿舍1L8B_第九宿舍1L8B_95000.csv
第九宿舍1LB1A_第九宿舍1LB1A_0.csv
./Dataset/20230118/building/第九宿舍1LB1A/第九宿舍1LB1A_第九宿舍1LB1A_0.csv
第九宿舍1LB1A_第九宿舍1LB1A_10000.csv
./Dataset/20230118/building/第九宿舍1LB1A/第九宿舍1

...

育成中心EMP2_育成中心EMP2_435000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_435000.csv
育成中心EMP2_育成中心EMP2_440000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_440000.csv
育成中心EMP2_育成中心EMP2_445000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_445000.csv
育成中心EMP2_育成中心EMP2_45000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_45000.csv
育成中心EMP2_育成中心EMP2_450000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_450000.csv
育成中心EMP2_育成中心EMP2_455000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_455000.csv
育成中心EMP2_育成中心EMP2_460000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_460000.csv
育成中心EMP2_育成中心EMP2_465000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_465000.csv
育成中心EMP2_育成中心EMP2_470000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_470000.csv
育成中心EMP2_育成中心EMP2_475000.csv
./Dataset/20230118/dormitory/育成中心EMP2/育成中心EMP2_育成中心EMP2_475000.csv
育成中心EMP2_育成中心EMP2_480000.csv
./Dataset/202

育成中心EMR_育成中心EMR_20000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_20000.csv
育成中心EMR_育成中心EMR_200000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_200000.csv
育成中心EMR_育成中心EMR_205000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_205000.csv
育成中心EMR_育成中心EMR_210000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_210000.csv
育成中心EMR_育成中心EMR_215000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_215000.csv
育成中心EMR_育成中心EMR_220000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_220000.csv
育成中心EMR_育成中心EMR_225000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_225000.csv
育成中心EMR_育成中心EMR_230000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_230000.csv
育成中心EMR_育成中心EMR_235000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_235000.csv
育成中心EMR_育成中心EMR_240000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_240000.csv
育成中心EMR_育成中心EMR_245000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_245000.csv
育

育成中心EMR_育成中心EMR_625000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_625000.csv
育成中心EMR_育成中心EMR_630000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_630000.csv
育成中心EMR_育成中心EMR_635000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_635000.csv
育成中心EMR_育成中心EMR_640000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_640000.csv
育成中心EMR_育成中心EMR_645000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_645000.csv
育成中心EMR_育成中心EMR_65000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_65000.csv
育成中心EMR_育成中心EMR_650000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_650000.csv
育成中心EMR_育成中心EMR_655000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_655000.csv
育成中心EMR_育成中心EMR_660000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_660000.csv
育成中心EMR_育成中心EMR_665000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_665000.csv
育成中心EMR_育成中心EMR_670000.csv
./Dataset/20230118/dormitory/育成中心EMR/育成中心EMR_育成中心EMR_670000.csv
育

育成中心MP1_育成中心MP1_375000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_375000.csv
育成中心MP1_育成中心MP1_380000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_380000.csv
育成中心MP1_育成中心MP1_385000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_385000.csv
育成中心MP1_育成中心MP1_390000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_390000.csv
育成中心MP1_育成中心MP1_395000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_395000.csv
育成中心MP1_育成中心MP1_40000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_40000.csv
育成中心MP1_育成中心MP1_400000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_400000.csv
育成中心MP1_育成中心MP1_405000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_405000.csv
育成中心MP1_育成中心MP1_410000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_410000.csv
育成中心MP1_育成中心MP1_415000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_415000.csv
育成中心MP1_育成中心MP1_420000.csv
./Dataset/20230118/dormitory/育成中心MP1/育成中心MP1_育成中心MP1_420000.csv
育

教學一錧L2.csv
教學一錧L3.csv
教學一錧L4.csv
教學一錧MPA-7.csv
教學一錧MRA.csv
教學一錧MVCB.csv
汙水處理廠EMP1.csv
汙水處理廠EMP2.csv
汙水處理廠HT1.csv
汙水處理廠HT2.csv
汙水處理廠HTM.csv
汙水處理廠MP1.csv
第九宿舍1L1A.csv
第九宿舍1L1B.csv
第九宿舍1L2A.csv
第九宿舍1L2B.csv
第九宿舍1L3A.csv
第九宿舍1L3B.csv
第九宿舍1L4A.csv
第九宿舍1L4B.csv
第九宿舍1L5A.csv
第九宿舍1L5B.csv
第九宿舍1L6A.csv
第九宿舍1L6B.csv
第九宿舍1L7A.csv
第九宿舍1L7B.csv
第九宿舍1L8A.csv
第九宿舍1L8B.csv
第九宿舍1LB1A.csv
第九宿舍1LRH.csv
第九宿舍1P1A.csv
第九宿舍1P1B.csv
第九宿舍1P2A.csv
第九宿舍1P2B.csv
第九宿舍1P3A.csv
第九宿舍1P3B.csv
第九宿舍1P4A.csv
第九宿舍1P4B.csv
第九宿舍1P5A.csv
第九宿舍1P5B.csv
第九宿舍1P6A.csv
第九宿舍1P6B.csv
第九宿舍1PK.csv
第九宿舍ACP&SC.csv
第九宿舍HT1.csv
第九宿舍HT2.csv
第九宿舍HT3.csv
第九宿舍HT4.csv
第九宿舍MP1-3.csv
第九宿舍PK1.csv
第十宿舍1LA.csv
第十宿舍1LB.csv
第十宿舍1PA.csv
第十宿舍1PB.csv
第十宿舍2L1.csv
第十宿舍2L2.csv
第十宿舍ACMP.csv
第十宿舍BLA.csv
第十宿舍BLB.csv
第十宿舍BPA.csv
第十宿舍BPB.csv
第十宿舍EMP.csv
第十宿舍HRP.csv
第十宿舍MPA.csv
第十宿舍MVCB.csv
第十宿舍PW.csv
第十宿舍RLA.csv
第十宿舍RLB.csv
第十宿舍SLA.csv
第十宿舍VCB3.csv
管理學院經世館HT2.csv
管理學院經世館HT3.csv
管理學院經世館HT4.csv
管理學院經世館HTM.csv
管理學院經世館L1.csv
管理學院經世館L2.csv
管理學院經世館L216.csv
管理學院經世館L3.

和氣象資料合併

In [6]:
# # 和氣象資料合併
# for filename in os.listdir(f"./Dataset/20230118/merge"): 
#     Summary = pd.read_csv(f'./Dataset/20230118/merge/{filename}')
#     sales1 = Summary
#     sales2 =  pd.read_csv('./Dataset/solar_汙水廠(history).csv')
#     sales1['TIME_TO_INTERVAL'] = pd.to_datetime(sales1['TIME_TO_INTERVAL'])
#     sales1['Date'] = sales1['TIME_TO_INTERVAL'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))
#     sales1['Date'] = sales1['TIME_TO_INTERVAL'].dt.date
#     sales1['Hour'] = sales1['TIME_TO_INTERVAL'].dt.hour
#     sales2['TIME_TO_INTERVAL'] = pd.to_datetime(sales2['TIME_TO_INTERVAL'])
#     sales2['Date'] = sales2['TIME_TO_INTERVAL'].dt.date.apply(lambda x: x.strftime('%Y-%m-%d'))
#     sales2['Date'] = sales2['TIME_TO_INTERVAL'].dt.date
#     sales2['Hour'] = sales2['TIME_TO_INTERVAL'].dt.hour


#     sales2 = sales2[['ApparentTemperature(pred)[CWB]','Temperature(pred)[CWB]','RelativeHumidity(pred)[CWB]','FeelsLikeTemperature(pred)[IBM]','Temperature(pred)[IBM]','RelativeHumidity(pred)[IBM]','FeelsLikeTemperature(pred)[OWM]','Temperature(pred)[OWM]','RelativeHumidity(pred)[OWM]','Date','Hour']]
#     sales2.rename(columns = {'ApparentTemperature(pred)[CWB]':'ApparentTemperature(pred)_CWB','Temperature(pred)[CWB]':'Temperature(pred)_CWB','RelativeHumidity(pred)[CWB]':'RelativeHumidity(pred)_CWB'}, inplace = True)
#     sales2.rename(columns = {'FeelsLikeTemperature(pred)[IBM]':'ApparentTemperature(pred)_IBM','Temperature(pred)[IBM]':'Temperature(pred)_IBM','RelativeHumidity(pred)[IBM]':'RelativeHumidity(pred)_IBM'}, inplace = True)
#     sales2.rename(columns = {'FeelsLikeTemperature(pred)[OWM]':'ApparentTemperature(pred)_OWM','Temperature(pred)[OWM]':'Temperature(pred)_OWM','RelativeHumidity(pred)[OWM]':'RelativeHumidity(pred)_OWM'}, inplace = True)
#     #         print(sales2)

#     columns = ['Date','Hour']
#     res = pd.merge(sales1,sales2, on=columns ) 
#     res = res.sort_values(by=['Date','Hour'])
#     res.to_csv(f'./Dataset/20230118/final/{filename}.csv', index=False)

In [ ]:
# concate
# for filename in os.listdir(f"./20220919/merge"): 
#     sales1 =  pd.read_csv(f'./old_temp/{filename}')
#     sales2 =  pd.read_csv(f'./20220919/merge/{filename}')
#     merge_data = pd.concat([sales1,sales2], axis=0, ignore_index=True)
#     print(merge_data.head(10))
#     print(merge_data.tail(10))
#     merge_data = merge_data.reset_index()
#     merge_data.to_csv(f'./temp/{filename}', index=False)